### Lendo o arquivo - CSV

In [39]:
import zipfile
import pandas as pd

# Setando as configuracoes do pandas ao visualizar dados
pd.set_option('display.max_columns', None)

with zipfile.ZipFile("../01 - Banco de dados/netflix_titles.csv.zip") as myzip:
    with myzip.open(myzip.namelist()[0]) as mycsv:
        df = pd.read_csv(mycsv)
        df = df.convert_dtypes()


### Salvando dados no banco de dados

In [63]:
from sqlalchemy import URL, create_engine
from IPython import get_ipython
import sys

url_object = URL.create(
    "postgresql+psycopg2",
    username="analista",
    password="32TSb+@tL955",  # plain (unescaped) text
    host="localhost",
    database="netflix",
)

engine = create_engine(url_object)

try:
    with engine.begin() as conn: pass
    print("✅ Conectado com sucesso")
except:
    print("⚠️ Erro na conexao com o banco")
    if not get_ipython(): sys.exit()



✅ Conectado com sucesso


In [ ]:
kwargs={
    "con":engine,
    "index":False,
    "if_exists":"replace",
    "schema":"netflix",
}

df.to_sql("netflix_titles", **kwargs)


787